# Access non-IPv6 services (i.e. GitHub) from IPv6 FABRIC nodes.



## Import the FABLlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

-----------------------------------  --------------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /Users/pruth/work/fabric_config/fabric_token.json
project_id                           990d8a8b-7e50-4d13-a3be-0f133ffa8653
bastion_username                     pruth_0031379841
bastion_key_filename                 /Users/pruth/work/fabric_config/fabric_bastion_key
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /Users/pruth/work/fabric_config/slice-public-key
slice_private_key_file               /Users/pruth/work/fabric_config/slice-private-key
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  -------------------

## Create a Node

Avoid IPv4 sites


In [2]:
try:
    #Create a slice
    slice = fablib.new_slice(name="MySlice")

    # Add a node
    slice.add_node(name="Node1", avoid['TACC','MAX'])

    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

SyntaxError: positional argument follows keyword argument (1491017880.py, line 6)

### Print the node


In [ ]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Upload and Exeute the NAT64 Script

In [ ]:
from ipaddress import ip_address, IPv6Address

try:
    node = slice.get_node(name="Node1")     
    
    #If the node is an IPv6 Node then configure NAT64
    if type(ip_address(node.get_management_ip())) is IPv6Address:
        node.upload_file('nat64.sh','nat64.sh')
        
        stdout, stderr = node.execute(f'chmod +x nat64.sh && ./nat64.sh')
        print(stdout)
        print(stderr)
        
    #Access non-IPv6 Services
    stdout, stderr = node.execute(f'sudo yum install -y -q git && git clone https://github.com/fabric-testbed/jupyter-examples.git')
    print(stdout)
    print(stderr)
    
    stdout, stderr = node.execute(f'ls jupyter-examples')
    print(stdout)
    print(stderr)
     
        
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")